loading required libraries

In [1]:
import pandas as pd
pd.set_option('display.max_colwidth', 5000)
from dotenv import load_dotenv
import json
from src.pdf import create_pdf
from src.gpt import evaluate_gpt
from src.utils import (
    load_preprocess_data,
    extract_all_diseases_per_department,
    filterDepartment,
    getDepartmentStatistics,
    run_prediction,
    evaluate_results)
load_dotenv()
from prompts import single_shot_disease_only_prompt,open_ended__top4_prompt,with_reasons_prompt

importing the dataset

In [2]:
filePath="dataset/clinicallab/data_en.json"
df=load_preprocess_data(filePath)



number of total cases are 1500

each case have the following fields ['id', 'clinical_case_uid', 'language', 'clinical_department', 'principal_diagnosis', 'preliminary_diagnosis', 'diagnostic_basis', 'differential_diagnosis', 'treatment_plan', 'clinical_case_summary', 'imageological_examination', 'laboratory_examination', 'pathological_examination', 'therapeutic_principle']

number of departments available are 24

 all the departments available are
clinical_department
orthopedics department                              100
anus and intestine surgical department              100
hepatobiliary and pancreas surgical department       99
urinary surgical department                          90
endocrinology department                             80
gynecology department                                80
otolaryngology head and neck surgical department     80
neurology department                                 80
thoracic surgical department                         70
respiratory medicine de

In [3]:
def extract_disease_names(diagnoses):
    disease_names = []
    for diagnosis in diagnoses:
        disease_name = diagnosis.split(':')[0]  # Split and get the disease name
        if len(disease_name) < 40:  # Check if the name is less than 40 characters
            disease_names.append(disease_name)
    return disease_names

# Apply the function to filter and transform the 'differential-diagnosis' field
df['differential_diagnosis'] = df['differential_diagnosis'].apply(extract_disease_names)

# Filter rows where the number of elements in 'differential-diagnosis' is greater than 1
filtered_df = df[df['differential_diagnosis'].apply(lambda x: len(x) > 1)]
filtered_df['differential_diagnosis'] = filtered_df['differential_diagnosis'] + filtered_df['principal_diagnosis'].apply(lambda x: [x] if x else [])

/var/tmp/ipykernel_748725/501556314.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['differential_diagnosis'] = filtered_df['differential_diagnosis'] + filtered_df['principal_diagnosis'].apply(lambda x: [x] if x else [])


In [4]:
print(len(df))
# filtered_df = df[df['differential_diagnosis'].apply(lambda x: len(x) > 2)]
print(len(filtered_df))

1500
1181


In [5]:
filtered_df.clinical_department.value_counts().sort_values(ascending=False)

clinical_department
anus and intestine surgical department              92
hepatobiliary and pancreas surgical department      80
endocrinology department                            78
neurology department                                77
urinary surgical department                         73
gastroenterology department                         70
respiratory medicine department                     65
gynecology department                               62
otolaryngology head and neck surgical department    62
pediatrics department                               57
gastrointestinal surgical department                52
thoracic surgical department                        52
nephrology department                               51
neurosurgery department                             47
thyroid surgical department                         43
cardiac surgical department                         40
breast surgical department                          39
cardiovascular medicine department           

In [6]:
departments=["nephrology department",
             "gynecology department",
            "endocrinology department", 
            "neurology department",
             "pediatrics department",
             "cardiac surgical department",                          
             "gastrointestinal surgical department",
             "respiratory medicine department",
             "gastroenterology department",
             "urinary surgical department",
             "hepatobiliary and pancreas surgical department",
             "hematology department"
            ]
models = ["gpt-4","llama3.1","gemma2", "phi3:14b", "mistral-nemo"]

In [7]:
filtered_df = filtered_df[filtered_df['clinical_department'].isin(departments)]
print(len(filtered_df))

729


In [8]:
729/3

243.0

filtering the dataset

In [ ]:


# type="with_reasons"
# prompt=with_reasons_prompt
type="single_shot"
prompt=single_shot_disease_only_prompt
# type="open_ended"
# # prompt=open_ended__top4_prompt# 
# run_prediction(df,prompt,departments,models=models,type=type,skip=5)

In [ ]:
# file_path="results/nephrology department_single_shot_without_sorted_result_findings.json"
# with open(file_path, "r") as file:
#     data = json.load(file)
# evaluate_results(data)

In [ ]:

import os
import json
from src.utils import evaluate_results
type="single_shot"
all_files=os.listdir("results")
all_results={}
for file in all_files:
    if type in file:
        file_path=os.path.join("results",file)
        department_name=file.split("_")[0]
        with open(file_path, "r") as file:
            data = json.load(file)
            results=evaluate_results(data)
        all_results[department_name]=results
        

In [ ]:
all_results